# Gender inference attack in observation phase

1. Data loading

In [22]:
import pandas as pd

# ratings
ratings_df = pd.read_csv('data/ratings.tsv', header=None, sep='\t')
ratings_df.rename(columns={0: 'uid', 1: 'movie_id', 2: 'rating', 3: 'timestamp'}, inplace=True)
ratings_df.set_index('id')
ratings_df.head(5)

KeyError: "None of ['id'] are in the columns"

In [12]:
# users
user_df = pd.read_csv('data/users.tsv', header=None, sep='\t')
user_df.head(5)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


filter out user attributes (mantain only id and gender)

In [23]:
user_gender_df = user_df.iloc[:, : 2]
user_gender_df.rename(columns={0: 'uid', 1: 'gender'}, inplace=True)
user_gender_df.set_index('uid')
user_gender_df.head(5)

,uid,gender
0,1,F
1,2,M
2,3,M
3,4,M
4,5,M


2. join datasets

In [36]:
joined_df = pd.merge(user_gender_df, ratings_df, on='uid')
joined_df.head(5)

,uid,gender,movie_id,rating,timestamp
0,1,F,1193,5,978300760
1,1,F,661,3,978302109
2,1,F,914,3,978301968
3,1,F,3408,4,978300275
4,1,F,2355,5,978824291


3. format dataset for classification task

In [38]:
joined_df.rename(columns={'gender': 'class'}, inplace=True)
joined_df['class'].replace('F', 0,inplace=True)
joined_df['class'].replace('M', 1,inplace=True)
joined_df['class'].unique()

array([0, 1])

4. training/test set splitting

In [40]:
from sklearn.model_selection import train_test_split

dff = joined_df
x = dff.iloc[:, dff.columns != 'class']
y = dff.iloc[:, dff.columns == 'class']
X_train, X_test, y_train, y_test = train_test_split (x,y, test_size = 0.3, random_state = 0)

5. classify (random forest)

In [43]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=100)

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

/var/folders/0r/m1t8hv6s7l9bsyqckwntb9qm0000gn/T/ipykernel_3345/918009877.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train,y_train)


6. evaluation metrics (random forest)

In [44]:
from sklearn.metrics import recall_score, precision_score, f1_score

recall_acc = recall_score(y_test,y_pred)
precision_acc = precision_score(y_test, y_pred)
f1_acc = f1_score(y_test, y_pred)
print("Recall: " + str(recall_acc) +"\nPrecision: " + str(precision_acc) +"\nF1_score: " + str(f1_acc))

Recall: 0.9971800357136543
Precision: 0.986920156083221
F1_score: 0.9920235687274646


7. classification (logistic regretion)

In [45]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
pred = logisticRegr.predict(X_test)

/Users/mauromarini/PycharmProjects/privacy_and_fairness_in_recsys/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


8. evaluation metrics (Logistic regression)

In [47]:
recall_acc = recall_score(pred,y_pred)
precision_acc = precision_score(pred, y_pred)
f1_acc = f1_score(pred, y_pred)
print("Recall: " + str(recall_acc) +"\nPrecision: " + str(precision_acc) +"\nF1_score: " + str(f1_acc))

Recall: 0.7618266830632234
Precision: 1.0
F1_score: 0.8648145591014246
